In [2]:
from datetime import timedelta
import warnings
warnings.filterwarnings("ignore")

import pandas as pd

from sklearn.preprocessing import StandardScaler

from tqdm import tqdm

In [3]:
data = pd.read_parquet('data/new_data/transactions.parquet')

In [4]:
print(data['user_id'].value_counts()) #sorted by number of iterations

a627f94faada4a50a6fb58b0554719d2    2972
3f3b5f6cbe7a41cc91fcd76cb796cd78    2952
fbd79611310047d6886129816d4d582e    2950
a86e8e61e76f4390a072e191395963ba    2946
3ff140e8aa1d471e98f4ed43b5da6c20    2931
                                    ... 
d593c24cccbd42ee824d1b3f748c6146     501
20d20678b7d340cda503aa78aa63c958     501
82dca6d27f1644209ed78552085800a8     501
ad3e7ce8481041cc816cf690c901aa9b     501
0bc9d468c9914bd09950e45eda0399fe     501
Name: user_id, Length: 22533, dtype: int64


Sort dataset by users and inside each user by time of transaction

In [5]:
data_srt = data.sort_values(['user_id','transaction_dttm']).reset_index()

In [6]:
data_srt.head(20)

,index,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm
0,0,000932580e404dafbecd5916d4640938,5411,48,-361.072300,2020-08-03 08:05:23
1,1,000932580e404dafbecd5916d4640938,5499,48,-137.313980,2020-08-05 01:27:40
2,2,000932580e404dafbecd5916d4640938,5499,48,-138.849810,2020-08-05 03:28:11
3,3,000932580e404dafbecd5916d4640938,4829,48,-309.476530,2020-08-06 00:36:29
4,4,000932580e404dafbecd5916d4640938,5411,48,-133.473700,2020-08-09 00:30:13
5,5,000932580e404dafbecd5916d4640938,5814,48,-63.414547,2020-08-09 01:18:28
6,6,000932580e404dafbecd5916d4640938,4789,48,-1640.281600,2020-08-09 23:43:24
7,7,000932580e404dafbecd5916d4640938,5411,48,-1497.890500,2020-08-10 03:26:58
8,8,000932580e404dafbecd5916d4640938,5814,48,-177.117340,2020-08-24 19:28:46
9,9,000932580e404dafbecd5916d4640938,5411,48,-95.083080,2020-08-27 01:46:31


In [9]:
start_user = data_srt.iloc[0]['user_id']
new_col = []
index = 0
start_time = data_srt.iloc[0]['transaction_dttm']
count = 0

for i in tqdm(range(len(data_srt))):
    cur_date = data_srt.iloc[i]['transaction_dttm']
    cur_user = data_srt.iloc[i]['user_id']
    count += 1
    if ((pd.to_datetime(cur_date)>pd.to_datetime(start_time)+timedelta(days=7))&(count>40))or(cur_user!=start_user): #suppose data is sorted
        count = 1
        index+=1
        start_time = cur_date #one_user.iloc[i]['transaction_dttm']
        start_user = cur_user
    new_col.append(index)

100%|██████████| 19821910/19821910 [1:10:21<00:00, 4694.93it/s]


In [12]:
data_srt['sample_label'] = pd.DataFrame(new_col)

In [16]:
data_srt = pd.read_parquet('labeled_sequences.parquet')

In [18]:
data_srt['mcc_code'].mean()

5400.251879057064

In [19]:
data_srt['mcc_code'].std()

1212.0683669438051

In [13]:
data_srt['sample_label'].value_counts()

243995    420
194428    270
208217    253
243997    252
243996    243
         ... 
368773      1
325605      1
193439      1
70200       1
330586      1
Name: sample_label, Length: 495834, dtype: int64

In [14]:
data_srt['sample_label'].nunique()

495834

In [27]:
data_srt[2558:2565]  #стык юзеров

,index,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm,sample_label
2558,2558,000b29acb6bd44f99473c1be5ca28f3c,5411,48,-35.813260,2021-07-29 05:50:09,64
2559,2559,000b29acb6bd44f99473c1be5ca28f3c,5814,48,-340.907500,2021-07-29 06:44:00,65
2560,2560,000b29acb6bd44f99473c1be5ca28f3c,5814,48,-184.804840,2021-07-29 08:32:52,65
2561,2561,000b29acb6bd44f99473c1be5ca28f3c,5921,48,-83.516490,2021-07-31 16:44:39,65
2562,2562,000c5327d99941fe934169838c65b92c,5422,48,-1923.653600,2020-07-31 19:56:00,66
2563,2563,000c5327d99941fe934169838c65b92c,5411,48,-794.831240,2020-07-31 19:58:00,66
2564,2564,000c5327d99941fe934169838c65b92c,5411,48,-41.320156,2020-07-31 20:49:00,66


In [3]:
data_srt = pd.read_parquet('labeled_sequences.parquet')

In [4]:
data_srt.drop(index=data_srt[data_srt['currency_rk'] != 48].index, inplace=True)

In [5]:
data_srt['count'] = data_srt.groupby('sample_label')['sample_label'].transform('count')
data_srt = data_srt[(data_srt['count'] >= 40) & (data_srt['count'] <= 120)]
data_srt.drop(columns=['count'], inplace=True)
data_srt

,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm,sample_label
0,000932580e404dafbecd5916d4640938,5411,48,-361.072300,2020-08-03 08:05:23,0
1,000932580e404dafbecd5916d4640938,5499,48,-137.313980,2020-08-05 01:27:40,0
2,000932580e404dafbecd5916d4640938,5499,48,-138.849810,2020-08-05 03:28:11,0
3,000932580e404dafbecd5916d4640938,4829,48,-309.476530,2020-08-06 00:36:29,0
4,000932580e404dafbecd5916d4640938,5411,48,-133.473700,2020-08-09 00:30:13,0
...,...,...,...,...,...,...
19821872,fffb051877b949f49bed6538393a06ae,5411,48,-84.112480,2021-07-18 09:47:30,495832
19821873,fffb051877b949f49bed6538393a06ae,5411,48,-196.376100,2021-07-18 22:03:37,495832
19821874,fffb051877b949f49bed6538393a06ae,5411,48,-113.904564,2021-07-19 01:47:07,495832
19821875,fffb051877b949f49bed6538393a06ae,5411,48,-64.439610,2021-07-19 06:09:04,495832


In [6]:
data_srt['anomaly'] = 0

In [7]:
def check_by_currency_rk(pd_data):
    pd_data.loc[pd_data['currency_rk'] != 48, 'anomaly'] = 1

def check_by_quantiles(pd_data):
    for mcc in tqdm(pd_data['mcc_code'].unique()):
        pos_pos = pd_data[(pd_data['mcc_code'] == mcc) & (pd_data['transaction_amt'] > 0)]['transaction_amt'].quantile(.999)
        pos_neg = pd_data[(pd_data['mcc_code'] == mcc) & (pd_data['transaction_amt'] < 0)]['transaction_amt'].quantile(.01)

        answ = pd_data[pd_data['mcc_code'] == mcc]['transaction_amt'].apply(lambda x: int(x > pos_pos or x < pos_neg))
        pd_data.loc[answ.index, 'anomaly'] = answ

In [8]:
check_by_quantiles(data_srt)

100%|██████████| 372/372 [01:49<00:00,  3.39it/s]


In [9]:
data_srt['anomaly'].sum()

176927

In [10]:
transactions_neg_scaled = StandardScaler().fit_transform(
    data_srt[data_srt['transaction_amt'] < 0]['transaction_amt'].to_numpy().reshape(-1, 1))
transactions_pos_scaled = StandardScaler().fit_transform(
    data_srt[data_srt['transaction_amt'] >= 0]['transaction_amt'].to_numpy().reshape(-1, 1))

In [11]:
data_srt['transaction_amt_scaled'] = 0
data_srt.loc[data_srt['transaction_amt'] < 0, 'transaction_amt_scaled'] = transactions_neg_scaled
data_srt.loc[data_srt['transaction_amt'] >= 0, 'transaction_amt_scaled'] = transactions_pos_scaled

In [12]:
sums = data_srt.groupby(['sample_label'])['anomaly'].sum()

In [14]:
anomaly_count = 0
normal_count = 0

for label in tqdm(data_srt['sample_label'].unique()):
    seq = data_srt[data_srt['sample_label'] == label][['user_id', 'mcc_code', 'transaction_amt_scaled', 'transaction_dttm']]
    anchor = pd.to_datetime(seq.iloc[0]['transaction_dttm'])
    seq['time'] = seq['transaction_dttm'].apply(lambda x: int((pd.to_datetime(x) - anchor).total_seconds()))
    seq.drop(columns=['transaction_dttm'], inplace=True)
    seq.rename(columns={'mcc_code': 'mcc', 'transaction_amt_scaled': 'amt'}, inplace=True)

    if sums.loc[label] > 0:
        seq.to_csv(f'data/new_data/anomaly/{anomaly_count}.csv', index=False)
        anomaly_count += 1
    else:
        seq.to_csv(f'data/new_data/normal/{normal_count}.csv', index=False)
        normal_count += 1

100%|██████████| 450930/450930 [4:30:27<00:00, 27.79it/s]  


In [15]:
pd.read_csv('data/new_data/anomaly/0.csv')

,user_id,mcc,amt,time
0,000932580e404dafbecd5916d4640938,6011,-0.050151,0
1,000932580e404dafbecd5916d4640938,5411,0.155519,7783
2,000932580e404dafbecd5916d4640938,5719,-1.701420,50978
3,000932580e404dafbecd5916d4640938,5814,0.206527,64497
4,000932580e404dafbecd5916d4640938,4829,-0.436117,86319
5,000932580e404dafbecd5916d4640938,4215,0.152970,142525
6,000932580e404dafbecd5916d4640938,5499,0.212159,154470
7,000932580e404dafbecd5916d4640938,5814,0.214666,224079
8,000932580e404dafbecd5916d4640938,5411,0.190913,240349
9,000932580e404dafbecd5916d4640938,5814,0.218662,397551
